Use this file as a template to copy and build notebooks

In [1]:
#@title Mount drive and load libraries
%%capture
!pip3 install torch torchvision torchaudio transformers sentencepiece accelerate --extra-index-url https://download.pytorch.org/whl/cu116
!python -m pip install ankh

from transformers import BertForMaskedLM, BertTokenizer, pipeline
from transformers import AutoModel, AutoTokenizer
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from google.colab import drive
import torch, os, pandas as pd, random, matplotlib.pyplot as plt, numpy as np
import seaborn as sns
import time, random, re, shutil
import ankh
from statistics import mean

from google.colab import drive

drive.mount('/content/drive/')
path = '/content/drive/MyDrive/msc-project-mbalmf01'
os.chdir(path)

In [2]:
#@title Read in MAPT scores and ScFv sequences
df = pd.read_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230813_human_paired_seqs_MAPT_scores.csv')
l = [pd.read_csv(f'/content/drive/MyDrive/msc-project-mbalmf01/all_paired/{i}') for i in os.listdir('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/') if '230811_human_paired_seqs_MAPT_ankh' in i]
scfv = pd.concat(l)
ankh_embeddings = scfv.iloc[:,:-1]

labels = df['MAPSS IgG1 pH7.4'].to_numpy()

X_train = ankh_embeddings.iloc[:45000, :].to_numpy()
y_train = labels[:45000]

X_val = ankh_embeddings.iloc[45000:, :].to_numpy()
y_val = labels[45000:]

training_labels_mean = mean(y_train)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device: {}".format(device))

In [3]:
from torch.utils.data import Dataset, DataLoader

class ProteinDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Assuming embeddings is a list/tensor of embeddings and labels is a list/tensor of labels
dataset = ProteinDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [8]:
#Create the Regression Head.
regression_model = ankh.ConvBertForRegression(input_dim=X_train.shape[-1],
                                              nhead=4,
                                              hidden_dim=384,
                                              num_hidden_layers=1,
                                              num_layers=1,
                                              kernel_size=7,
                                              dropout=0,
                                              pooling='max',
                                              training_labels_mean=training_labels_mean)

regression_model = regression_model.float()


In [9]:
import torch.optim as optim

# The model already calculates MSE loss internally, so you don't need an explicit loss function
optimizer = optim.Adam(regression_model.parameters(), lr=0.001)


In [10]:
num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0
    for batch_embeddings, batch_labels in dataloader:
        optimizer.zero_grad()
        batch_embeddings = batch_embeddings.float()
        batch_labels = batch_labels.float()
        outputs = regression_model(batch_embeddings, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader)}")


IndexError: ignored

In [11]:
print(batch_embeddings.shape)


torch.Size([32, 1536])
